In [1]:
import os
import sys
sys.dont_write_bytecode = True
sys.path.append("../")
sys.path.append("../../")

import warnings
warnings.filterwarnings("ignore") 

import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from functools import partial
import multiprocessing as mp


def load_data(benchmark, method, seed):
    print("\r", end="")
    filename = f'logs/samples-{benchmark}-{method}-seed{seed}.log'
    with open(filename, 'r') as f:
        total_lines = sum(1 for line in f)
        total_chunks = (total_lines - 1) // 1000 + 1
    # return pd.read_csv(filename, nrows=100)
    generator = tqdm(pd.read_csv(filename, chunksize=1000), desc=f'[Loading {filename}]', total=total_chunks, leave=False)
    return pd.concat([chunk for chunk in generator])
    

def compute_total_samples(benchmark, method, num_seeds):
    total_samples = []
    for seed in range(num_seeds):
        filename = f'logs/samples-{benchmark}-{method}-seed{seed}.log'
        with open(filename, 'r') as f:
            total_lines = sum(1 for line in f)
            total_samples.append(total_lines - 1)  # Exclude header
    return np.array(total_samples)


def compute_time_and_memory(benchmark, method, num_seeds):
    time = []
    memory = []

    with mp.Pool(processes=num_seeds) as pool:
        dfs = list(pool.map(partial(load_data, benchmark, method), [seed for seed in range(num_seeds)]))

    for seed in range(num_seeds):
        cumulative_time = dfs[seed]["time"].to_numpy()
        time.append(cumulative_time - np.concatenate(([0], cumulative_time[:-1])))
        memory.append(dfs[seed]["memory"].to_numpy() / (10 ** 9))

    return np.concatenate(time), np.concatenate(memory)

,time,chain,memory,theta[0][0],theta[0][1],theta[0][2],theta[0][3],theta[0][4],theta[0][5],theta[0][6],...,q_noise[199][0],q_noise[199][1],q_noise[199][2],q_noise[199][3],q_noise[199][4],q_noise[199][5],q_noise[199][6],q_noise[199][7],q_noise[199][8],q_noise[199][9]
0,0.378479,0,449024,19.074423,19.574392,17.353775,20.145325,19.879074,19.419796,19.376577,...,0.644945,0.601498,0.472842,0.515443,0.292436,0.531044,0.379930,0.363675,0.527291,0.364453
1,0.750350,0,465408,19.074423,19.574392,17.353775,20.145325,19.879074,19.419796,19.376577,...,0.644945,0.601498,0.472842,0.515443,0.292436,0.531044,0.379930,0.363675,0.527291,0.364453
2,1.119060,0,465408,19.074423,19.574392,17.353775,20.145325,19.879074,19.419796,19.376577,...,0.644945,0.601498,0.472842,0.515443,0.292436,0.531044,0.379930,0.363675,0.527291,0.364453
3,1.485285,0,465408,19.074423,19.574392,17.353775,20.145325,19.879074,19.419796,19.376577,...,0.644945,0.601498,0.472842,0.515443,0.292436,0.531044,0.379930,0.363675,0.527291,0.364453
4,1.858558,0,465408,19.073235,19.574764,17.353361,20.144772,19.880409,19.418795,19.377100,...,0.646301,0.600768,0.474074,0.515841,0.292443,0.531686,0.381371,0.363508,0.527675,0.365753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,37.226100,0,465408,19.068283,19.580276,17.363647,20.145338,19.882481,19.421577,19.380457,...,0.642598,0.584750,0.487310,0.512448,0.277266,0.522654,0.384321,0.376855,0.527813,0.382376
96,37.630332,0,465408,19.068283,19.580276,17.363647,20.145338,19.882481,19.421577,19.380457,...,0.642598,0.584750,0.487310,0.512448,0.277266,0.522654,0.384321,0.376855,0.527813,0.382376
97,38.043643,0,465408,19.068283,19.580276,17.363647,20.145338,19.882481,19.421577,19.380457,...,0.642598,0.584750,0.487310,0.512448,0.277266,0.522654,0.384321,0.376855,0.527813,0.382376
98,38.445994,0,465408,19.068283,19.580276,17.363647,20.145338,19.882481,19.421577,19.380457,...,0.642598,0.584750,0.487310,0.512448,0.277266,0.522654,0.384321,0.376855,0.527813,0.382376


In [ ]:
num_seeds = 8

benchmarks = ["hmm-ord1", "hmm-ord2", "nhmm-train", "nhmm-stock", "arm", "tcm"]
methods = ["seq", "plate", "vmarkov", "discHMM", "manual", "ours"]

if locals().get('total_samples_mean') is None \
    or locals().get('total_samples_std') is None \
    or locals().get('total_samples_ratio') is None:

    total_samples_mean = pd.DataFrame()
    total_samples_std = pd.DataFrame()

    for benchmark in tqdm(benchmarks):
        mean = []
        std = []

        for method in methods:
            try:
                total_samples = compute_total_samples(benchmark, method, num_seeds)
                mean.append(total_samples.mean())
                std.append(total_samples.std())

            except FileNotFoundError as e:
                mean.append(float('nan'))
                std.append(float('nan'))

        total_samples_mean = pd.concat([total_samples_mean, pd.DataFrame([mean])], axis=0)
        total_samples_std = pd.concat([total_samples_std, pd.DataFrame([std])], axis=0)

    total_samples_mean.index = benchmarks
    total_samples_mean.columns = methods
    total_samples_std.index = benchmarks
    total_samples_std.columns = methods

    total_samples_ratio = total_samples_mean.div(total_samples_mean['ours'], axis=0)


display(total_samples_mean.round(3))
display(total_samples_ratio.round(4))
display(total_samples_std.round(4))
display((1/total_samples_ratio).round(4))

print(total_samples_mean.to_latex(formatters=[lambda x: f"{x:.3f}"]*len(total_samples_mean.columns), na_rep="-", bold_rows=True))
print(total_samples_ratio.to_latex(formatters=[lambda x: f"{x:.4f}"]*len(total_samples_ratio.columns), na_rep="-", bold_rows=True))
print(total_samples_std.to_latex(formatters=[lambda x: f"{x:.4f}"]*len(total_samples_std.columns), na_rep="-", bold_rows=True))
print((1/total_samples_ratio).to_latex(formatters=[lambda x: f"{x:.4f}"]*len(total_samples_ratio.columns), na_rep="-", bold_rows=True))

: 

In [2]:
num_seeds = 8

benchmarks = ["hmm-ord1", "hmm-ord2", "nhmm-train", "nhmm-stock", "arm", "tcm"]
methods = ["seq", "plate", "vmarkov", "discHMM", "manual", "ours"]

if locals().get('total_samples_mean') is None \
    or locals().get('total_samples_std') is None \
    or locals().get('total_samples_ratio') is None:

    total_time_mean = pd.DataFrame()
    total_time_std = pd.DataFrame()
    total_memory_mean = pd.DataFrame()
    total_memory_std = pd.DataFrame()

    for benchmark in tqdm(benchmarks):
        time_mean = []
        time_std = []
        memory_mean = []
        memory_std = []

        for method in methods:
            try:
                time, memory = compute_time_and_memory(benchmark, method, num_seeds)

                try:
                    time_mean.append(time.mean())
                except ZeroDivisionError:
                    time_mean.append(float('nan'))

                try:
                    time_std.append(time.std())
                except ZeroDivisionError:
                    time_std.append(float('nan'))

                try:
                    memory_mean.append(memory.mean())
                except ZeroDivisionError:
                    memory_mean.append(float('nan'))

                try:
                    memory_std.append(memory.std())
                except ZeroDivisionError:
                    memory_std.append(float('nan'))

            except FileNotFoundError as e:
                time_mean.append(float('nan'))
                time_std.append(float('nan'))
                memory_mean.append(float('nan'))
                memory_std.append(float('nan'))

        total_time_mean = pd.concat([total_time_mean, pd.DataFrame([time_mean])], axis=0)
        total_time_std = pd.concat([total_time_std, pd.DataFrame([time_std])], axis=0)
        total_memory_mean = pd.concat([total_memory_mean, pd.DataFrame([memory_mean])], axis=0)
        total_memory_std = pd.concat([total_memory_std, pd.DataFrame([memory_std])], axis=0)

    for l in [total_time_mean, total_time_std, total_memory_mean, total_memory_std]:
        l.index = benchmarks
        l.columns = methods

    total_time_ratio = total_time_mean.div(total_time_mean['ours'], axis=0)
    total_memory_ratio = total_memory_mean.div(total_memory_mean['ours'], axis=0)

  0%|          | 0/6 [00:00<?, ?it/s]

[Loading logs/samples-hmm-ord1-seq-seed0.log]:   0%|          | 0/1 [00:00<?, ?it/s]

[Loading logs/samples-hmm-ord1-seq-seed3.log]:   0%|          | 0/1 [00:00<?, ?it/s]

[Loading logs/samples-hmm-ord1-seq-seed7.log]:   0%|          | 0/1 [00:00<?, ?it/s]

[Loading logs/samples-hmm-ord1-seq-seed1.log]:   0%|          | 0/1 [00:00<?, ?it/s]

...

[Loading logs/samples-tcm-ours-seed3.log]:   0%|          | 0/9 [00:00<?, ?it/s]

[Loading logs/samples-tcm-ours-seed1.log]:   0%|          | 0/9 [00:00<?, ?it/s]

[Loading logs/samples-tcm-ours-seed5.log]:   0%|          | 0/9 [00:00<?, ?it/s]

[Loading logs/samples-tcm-ours-seed0.log]:   0%|          | 0/10 [00:00<?, ?it/s]

[Loading logs/samples-tcm-ours-seed2.log]:   0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
display(total_time_mean.round(3))
display(total_time_ratio.round(3))
display(total_time_std.round(3))

print(total_time_mean.to_latex(formatters=[lambda x: f"{x:.3f}"]*len(total_time_mean.columns), na_rep="-", bold_rows=True))
print(total_time_ratio.to_latex(formatters=[lambda x: f"{x:.3f}"]*len(total_time_ratio.columns), na_rep="-", bold_rows=True))
print(total_time_std.to_latex(formatters=[lambda x: f"{x:.3f}"]*len(total_time_std.columns), na_rep="-", bold_rows=True))

print(3600 / total_time_mean["ours"]["hmm-ord1"])

,seq,plate,vmarkov,discHMM,manual,ours
hmm-ord1,NaN,1.872,0.844,0.223,0.399,0.786
hmm-ord2,1868.995,2.826,2.734,NaN,1.238,1.545
nhmm-train,1232.830,245.348,NaN,NaN,2.512,3.738
nhmm-stock,510.519,52.833,NaN,NaN,0.560,1.439
arm,21.952,NaN,0.489,NaN,0.049,0.191
tcm,28.087,NaN,NaN,NaN,NaN,0.417


,seq,plate,vmarkov,discHMM,manual,ours
hmm-ord1,NaN,2.383,1.074,0.284,0.507,1.0
hmm-ord2,1209.649,1.829,1.769,NaN,0.801,1.0
nhmm-train,329.769,65.628,NaN,NaN,0.672,1.0
nhmm-stock,354.730,36.710,NaN,NaN,0.389,1.0
arm,114.806,NaN,2.557,NaN,0.256,1.0
tcm,67.277,NaN,NaN,NaN,NaN,1.0


,seq,plate,vmarkov,discHMM,manual,ours
hmm-ord1,NaN,0.093,0.020,0.016,0.020,0.027
hmm-ord2,84.462,0.176,0.100,NaN,0.067,0.096
nhmm-train,38.036,6.592,NaN,NaN,0.036,0.081
nhmm-stock,30.262,2.965,NaN,NaN,0.035,0.078
arm,0.902,NaN,0.069,NaN,0.004,0.018
tcm,0.859,NaN,NaN,NaN,NaN,0.027


\begin{tabular}{lrrrrrr}
\toprule
 & seq & plate & vmarkov & discHMM & manual & ours \\
\midrule
\textbf{hmm-ord1} & - & 1.872 & 0.844 & 0.223 & 0.399 & 0.786 \\
\textbf{hmm-ord2} & 1868.995 & 2.826 & 2.734 & - & 1.238 & 1.545 \\
\textbf{nhmm-train} & 1232.830 & 245.348 & - & - & 2.512 & 3.738 \\
\textbf{nhmm-stock} & 510.519 & 52.833 & - & - & 0.560 & 1.439 \\
\textbf{arm} & 21.952 & - & 0.489 & - & 0.049 & 0.191 \\
\textbf{tcm} & 28.087 & - & - & - & - & 0.417 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrr}
\toprule
 & seq & plate & vmarkov & discHMM & manual & ours \\
\midrule
\textbf{hmm-ord1} & - & 2.383 & 1.074 & 0.284 & 0.507 & 1.000 \\
\textbf{hmm-ord2} & 1209.649 & 1.829 & 1.769 & - & 0.801 & 1.000 \\
\textbf{nhmm-train} & 329.769 & 65.628 & - & - & 0.672 & 1.000 \\
\textbf{nhmm-stock} & 354.730 & 36.710 & - & - & 0.389 & 1.000 \\
\textbf{arm} & 114.806 & - & 2.557 & - & 0.256 & 1.000 \\
\textbf{tcm} & 67.277 & - & - & - & - & 1.000 \\
\bottomrule
\end{tabular}

\begin{

In [7]:
display(total_memory_mean.round(4))
display(total_memory_ratio.round(4))
display(total_memory_std.round(4))
display((100/total_memory_ratio).round(4))

print(total_memory_mean.to_latex(formatters=[lambda x: f"{x:.4f}"]*len(total_memory_mean.columns), na_rep="-", bold_rows=True))
print(total_memory_ratio.to_latex(formatters=[lambda x: f"{x:.4f}"]*len(total_memory_ratio.columns), na_rep="-", bold_rows=True))
print(total_memory_std.to_latex(formatters=[lambda x: f"{x:.4f}"]*len(total_memory_std.columns), na_rep="-", bold_rows=True))
print((100/total_memory_ratio).to_latex(formatters=[lambda x: f"{x:.4f}"]*len(total_memory_ratio.columns), na_rep="-", bold_rows=True))

,seq,plate,vmarkov,discHMM,manual,ours
hmm-ord1,NaN,0.7000,1.2663,0.2464,0.6681,1.3834
hmm-ord2,1.8299,0.8659,3.8962,NaN,2.0342,2.1895
nhmm-train,3.3428,2.8506,NaN,NaN,3.6922,3.7635
nhmm-stock,0.2446,0.0517,NaN,NaN,0.0344,0.0358
arm,0.0103,NaN,0.0090,NaN,0.0002,0.0022
tcm,0.0139,NaN,NaN,NaN,NaN,0.0005


,seq,plate,vmarkov,discHMM,manual,ours
hmm-ord1,NaN,0.5060,0.9153,0.1781,0.4830,1.0
hmm-ord2,0.8358,0.3955,1.7795,NaN,0.9291,1.0
nhmm-train,0.8882,0.7574,NaN,NaN,0.9811,1.0
nhmm-stock,6.8400,1.4457,NaN,NaN,0.9607,1.0
arm,4.7768,NaN,4.1773,NaN,0.0775,1.0
tcm,29.5086,NaN,NaN,NaN,NaN,1.0


,seq,plate,vmarkov,discHMM,manual,ours
hmm-ord1,NaN,0.0,0.0000,0.0,0.0,0.000
hmm-ord2,0.0,0.0,0.0000,NaN,0.0,0.000
nhmm-train,0.0,0.0,NaN,NaN,0.0,0.000
nhmm-stock,0.0,0.0,NaN,NaN,0.0,0.000
arm,0.0,NaN,0.0001,NaN,0.0,0.001
tcm,0.0,NaN,NaN,NaN,NaN,0.000


,seq,plate,vmarkov,discHMM,manual,ours
hmm-ord1,NaN,197.6221,109.2499,561.5698,207.0558,100.0
hmm-ord2,119.6522,252.8524,56.1951,NaN,107.6342,100.0
nhmm-train,112.5827,132.0229,NaN,NaN,101.9294,100.0
nhmm-stock,14.6198,69.1721,NaN,NaN,104.0959,100.0
arm,20.9345,NaN,23.9387,NaN,1290.3648,100.0
tcm,3.3888,NaN,NaN,NaN,NaN,100.0


\begin{tabular}{lrrrrrr}
\toprule
 & seq & plate & vmarkov & discHMM & manual & ours \\
\midrule
\textbf{hmm-ord1} & - & 0.7000 & 1.2663 & 0.2464 & 0.6681 & 1.3834 \\
\textbf{hmm-ord2} & 1.8299 & 0.8659 & 3.8962 & - & 2.0342 & 2.1895 \\
\textbf{nhmm-train} & 3.3428 & 2.8506 & - & - & 3.6922 & 3.7635 \\
\textbf{nhmm-stock} & 0.2446 & 0.0517 & - & - & 0.0344 & 0.0358 \\
\textbf{arm} & 0.0103 & - & 0.0090 & - & 0.0002 & 0.0022 \\
\textbf{tcm} & 0.0139 & - & - & - & - & 0.0005 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrr}
\toprule
 & seq & plate & vmarkov & discHMM & manual & ours \\
\midrule
\textbf{hmm-ord1} & - & 0.5060 & 0.9153 & 0.1781 & 0.4830 & 1.0000 \\
\textbf{hmm-ord2} & 0.8358 & 0.3955 & 1.7795 & - & 0.9291 & 1.0000 \\
\textbf{nhmm-train} & 0.8882 & 0.7574 & - & - & 0.9811 & 1.0000 \\
\textbf{nhmm-stock} & 6.8400 & 1.4457 & - & - & 0.9607 & 1.0000 \\
\textbf{arm} & 4.7768 & - & 4.1773 & - & 0.0775 & 1.0000 \\
\textbf{tcm} & 29.5086 & - & - & - & - & 1.0000 \\
\bottomrul